# **Código para módulo ESP32 receptor**

In [ ]:
import espnow
from network import WLAN
from machine import Pin
import struct
import time

# ----------------------------
# Configuración LED RGB (cátodo común)
# ----------------------------
LED_R = Pin(13, Pin.OUT)  # Rojo
LED_G = Pin(12, Pin.OUT)  # Verde
LED_B = Pin(14, Pin.OUT)  # Azul

def set_color(r, g, b):
    LED_R.value(r)
    LED_G.value(g)
    LED_B.value(b)

def off():
    set_color(1, 1, 1)

# ----------------------------
# MAC del transmisor
# ----------------------------
TRANSMITTER_MAC = b"\xcc\xdb\xa7\x97\xa9\xac"

# ----------------------------
# Inicializar WiFi en modo STA para ESP-NOW
# ----------------------------
wlan = WLAN()
wlan.active(True)

# ----------------------------
# Inicializar ESP-NOW
# ----------------------------
e = espnow.ESPNow()
e.active(True)

try:
    e.add_peer(TRANSMITTER_MAC)
except:
    pass

print("Receptor listo, esperando datos...")

# ----------------------------
# Variables para control de conexión
# ----------------------------
last_msg_time = time.ticks_ms()
alert_start = None

# ----------------------------
# Función para mostrar EMG en LED
# ----------------------------
def show_emg_rgb(value):
    if value < 2000:        # señal baja → azul
        set_color(1, 1, 0)
    else:                   # señal alta → verde
        set_color(1, 0, 1)

# ----------------------------
# Bucle principal
# ----------------------------
while True:
    host, msg = e.recv()

    if msg:  # Se recibió un mensaje
        last_msg_time = time.ticks_ms()
        alert_start = None
        emg_value = struct.unpack(">H", msg)[0]
        print("Valor EMG:", emg_value)
        show_emg_rgb(emg_value)

    else:
        elapsed = time.ticks_diff(time.ticks_ms(), last_msg_time)

        if elapsed > 60000:  # más de 60 seg sin señal → apagar
            off()

        elif elapsed > 2000:  # más de 2 seg sin señal → rojo intermitente
            if alert_start is None:
                alert_start = time.ticks_ms()
            # parpadeo rojo
            if (time.ticks_diff(time.ticks_ms(), alert_start) // 500) % 2 == 0:
                set_color(0, 1, 1)
            else:
                off()

    time.sleep_ms(50)